# Домашнее задание №2

**

## Часть 1. ML workflow (**всего 5 баллов**)

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

### Загрузим данные для работы. 

In [3]:
df = pd.read_csv("winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Будем решать задачу регрессии: необходимо предсказать качество вина на основе его характеристик

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("quality")

In [4]:
X = df.drop('quality', axis=1)
y = df['quality']

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [17]:
model = LinearRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [18]:
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"MSE на тренировочных данных: {mse_train:.4f}")
print(f"MSE на тестовых данных: {mse_test:.4f}")
print(f"R2 на тренировочных данных: {r2_train:.4f}")
print(f"R2 на тестовых данных: {r2_test:.4f}")

MSE на тренировочных данных: 0.4208
MSE на тестовых данных: 0.4112
R2 на тренировочных данных: 0.3612
R2 на тестовых данных: 0.3514


### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [19]:
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')
mean_cv_score = cv_scores.mean()

print(f"Среднее качество (R2) на кросс-валидации: {mean_cv_score:.4f}")
print(f"Оценки R2 на фолдах: {cv_scores}")

Среднее качество (R2) на кросс-валидации: 0.2900
Оценки R2 на фолдах: [0.13200871 0.31858135 0.34955348 0.369145   0.2809196 ]


### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1. 

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [9]:
from sklearn.linear_model import Lasso

alphas = np.arange(0.1, 1.1, 0.1)
param_grid = {'alpha': alphas}

lasso = Lasso()
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
             scoring='r2')

### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [10]:
best_alpha = grid_search.best_params_['alpha']
best_score = grid_search.best_score_

print(f"Наилучший параметр alpha: {best_alpha}")
print(f"Наилучшее качество (R2) на кросс-валидации: {best_score:.4f}")
print(f"Наилучший алгоритм: {grid_search.best_estimator_}")

Наилучший параметр alpha: 0.1
Наилучшее качество (R2) на кросс-валидации: 0.2182
Наилучший алгоритм: Lasso(alpha=0.1)


### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [11]:
best_model = grid_search.best_estimator_
y_test_pred_lasso = best_model.predict(X_test)
r2_test_lasso = r2_score(y_test, y_test_pred_lasso)

print(f"R2-score модели Lasso на тестовых данных: {r2_test_lasso:.4f}")

R2-score модели Lasso на тестовых данных: 0.2550


### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [20]:
from sklearn.preprocessing import StandardScaler

poly_pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2)),
    ('scaler', StandardScaler()),
    ('linear', LinearRegression())
])

cv_scores_poly = cross_val_score(poly_pipeline, X, y, cv=5, scoring='r2')
mean_cv_score_poly = cv_scores_poly.mean()

print(f"Среднее качество (R2) на кросс-валидации для модели с полиномами: {mean_cv_score_poly:.4f}")

Среднее качество (R2) на кросс-валидации для модели с полиномами: 0.2301


### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [21]:
poly_pipeline.fit(X_train, y_train)

y_train_pred_poly = poly_pipeline.predict(X_train)
y_test_pred_poly = poly_pipeline.predict(X_test)

mse_train_poly = mean_squared_error(y_train, y_train_pred_poly)
mse_test_poly = mean_squared_error(y_test, y_test_pred_poly)
r2_train_poly = r2_score(y_train, y_train_pred_poly)
r2_test_poly = r2_score(y_test, y_test_pred_poly)

print(f"MSE на тренировочных данных: {mse_train_poly:.4f}")
print(f"MSE на тестовых данных: {mse_test_poly:.4f}")
print(f"R2 на тренировочных данных: {r2_train_poly:.4f}")
print(f"R2 на тестовых данных: {r2_test_poly:.4f}")

MSE на тренировочных данных: 0.3618
MSE на тестовых данных: 0.4113
R2 на тренировочных данных: 0.4508
R2 на тестовых данных: 0.3514


### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

In [22]:
"""
1) Исходная модель (линейная регрессия без регуляризации) показала посредственное качество с R2 около 0.35-0.40. 
   Модель не сильно переобучена, так как она показала себя на тестовых данных даже лучше, чем на тренировочных

2) L1-регуляризация (Lasso) не привела к существенному улучшению качества модели. 
   Это может означать, что проблема не в переобучении, а в том, что линейная модель в принципе 
   не подходит для описания зависимостей в данных.

3) Добавление полиномиальных признаков второй степени незначительно улучшило качество модели, 
   повысив R2 до 0.45 на тренировочных данных. Однако это привело к переобучению, 
   так как качество на тестовых данных стало не стало лучше. Модель стала слишком сложной и 
   начала подстраиваться под шум в данных, а не выявлять истинные закономерности.
"""


'\n1) Исходная модель (линейная регрессия без регуляризации) показала посредственное качество с R2 около 0.35-0.40. \n   Модель не сильно переобучена, так как она показала себя на тестовых данных даже лучше, чем на тренировочных\n\n2) L1-регуляризация (Lasso) не привела к существенному улучшению качества модели. \n   Это может означать, что проблема не в переобучении, а в том, что линейная модель в принципе \n   не подходит для описания зависимостей в данных.\n\n3) Добавление полиномиальных признаков второй степени незначительно улучшило качество модели, \n   повысив R2 до 0.45 на тренировочных данных. Однако это привело к переобучению, \n   так как качество на тестовых данных стало не стало лучше. Модель стала слишком сложной и \n   начала подстраиваться под шум в данных, а не выявлять истинные закономерности.\n'

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

In [28]:
# Импортируем дополнительные библиотеки
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# 1. Предобработка данных
# Создадим новую копию данных
X_improved = df.drop('quality', axis=1).copy()
y_improved = df['quality'].copy()

# Создадим признаки взаимодействия, которые могут быть полезны для вина
X_improved['alcohol_sugar_ratio'] = X_improved['alcohol'] / (X_improved['residual sugar'] + 0.1)
X_improved['acid_balance'] = X_improved['fixed acidity'] / (X_improved['volatile acidity'] + 0.1)
X_improved['sulfur_balance'] = X_improved['free sulfur dioxide'] / (X_improved['total sulfur dioxide'] + 0.1)
X_improved['acidity_index'] = X_improved['fixed acidity'] * X_improved['pH']

# Логарифмические преобразования для признаков с неравномерным распределением
X_improved['log_chlorides'] = np.log1p(X_improved['chlorides'])
X_improved['log_sulphates'] = np.log1p(X_improved['sulphates'])
X_improved['log_alcohol'] = np.log1p(X_improved['alcohol'])

# Разделение на обучающую и тестовую выборки
X_train_imp, X_test_imp, y_train_imp, y_test_imp = train_test_split(X_improved, y_improved, test_size=0.3, random_state=42)

# 2. Создадим сложный пайплайн с выбором признаков и нелинейной моделью
complex_pipeline = Pipeline([
    ('scaler', RobustScaler()),  # Устойчивый к выбросам скейлер
    ('feature_select', SelectKBest(f_regression, k=10)),  # Выбираем лучшие признаки
    ('model', GradientBoostingRegressor(random_state=42))  # Градиентный бустинг
])

# 3. Настройка гиперпараметров с помощью RandomizedSearchCV
param_distributions = {
    'model__n_estimators': randint(50, 500),
    'model__max_depth': randint(3, 10),
    'model__min_samples_split': randint(2, 20),
    'model__learning_rate': uniform(0.01, 0.3),
    'model__subsample': uniform(0.6, 0.4),
    'feature_select__k': randint(5, 15)
}

random_search = RandomizedSearchCV(
    complex_pipeline,
    param_distributions=param_distributions,
    n_iter=50,
    cv=5,
    scoring='r2',
    random_state=42,
    n_jobs=10  # Используем все доступные ядра процессора
)

# Обучение модели
random_search.fit(X_train_imp, y_train_imp)

# 4. Оценка качества модели
best_model = random_search.best_estimator_
y_pred_train = best_model.predict(X_train_imp)
y_pred_test = best_model.predict(X_test_imp)

# Выведем метрики
r2_train_improved = r2_score(y_train_imp, y_pred_train)
r2_test_improved = r2_score(y_test_imp, y_pred_test)
mse_train_improved = mean_squared_error(y_train_imp, y_pred_train)
mse_test_improved = mean_squared_error(y_test_imp, y_pred_test)

print(f"R2 на тренировочных данных: {r2_train_improved:.4f}")
print(f"R2 на тестовых данных: {r2_test_improved:.4f}")
print(f"MSE на тренировочных данных: {mse_train_improved:.4f}")
print(f"MSE на тестовых данных: {mse_test_improved:.4f}")

# 5. Посмотрим важность признаков
feature_importances = pd.DataFrame(
    {'feature': X_train_imp.columns[best_model.named_steps['feature_select'].get_support()],
     'importance': best_model.named_steps['model'].feature_importances_}
).sort_values('importance', ascending=False)

print("Важность признаков:")
print(feature_importances)

# 6. Сравним с другими моделями для подтверждения результатов
models = {
    'Random Forest': RandomForestRegressor(n_estimators=200, random_state=42),
    'SVR': SVR(kernel='rbf', C=10, gamma='scale'),
    'KNN': KNeighborsRegressor(n_neighbors=5),
}

results = []
for name, model in models.items():
    pipeline = Pipeline([
        ('scaler', RobustScaler()),
        (name, model)
    ])
    pipeline.fit(X_train_imp, y_train_imp)
    r2_test = r2_score(y_test_imp, pipeline.predict(X_test_imp))
    results.append({'Модель': name, 'R2 на тесте': r2_test})

comparative_results = pd.DataFrame(results)
# Вместо append используем concat с новой строкой данных
new_row = pd.DataFrame([{'Модель': 'Градиентный бустинг (тюнинг)', 'R2 на тесте': r2_test_improved}])
comparative_results = pd.concat([comparative_results, new_row], ignore_index=True)


print("\nСравнение моделей:")
print(comparative_results.sort_values('R2 на тесте', ascending=False))

# 7. Дополнительно: Ансамблирование моделей для еще лучших результатов
from sklearn.ensemble import VotingRegressor

final_models = []
for name, model in models.items():
    final_models.append((name, Pipeline([('scaler', RobustScaler()), ('model', model)])))
final_models.append(('GradientBoosting', best_model))

ensemble = VotingRegressor(final_models)
ensemble.fit(X_train_imp, y_train_imp)
ensemble_r2 = r2_score(y_test_imp, ensemble.predict(X_test_imp))

print(f"\nИтоговое качество ансамбля моделей (R2): {ensemble_r2:.4f}")

R2 на тренировочных данных: 0.9297
R2 на тестовых данных: 0.4450
MSE на тренировочных данных: 0.0463
MSE на тестовых данных: 0.3519
Важность признаков:
                 feature  importance
13           log_alcohol    0.132628
6                alcohol    0.128770
0       volatile acidity    0.088976
3   total sulfur dioxide    0.081536
9         sulfur_balance    0.078014
8           acid_balance    0.075378
5              sulphates    0.072130
7    alcohol_sugar_ratio    0.067663
12         log_sulphates    0.063727
1            citric acid    0.051442
10         acidity_index    0.049625
4                density    0.048178
2              chlorides    0.031807
11         log_chlorides    0.030128

Сравнение моделей:
                         Модель  R2 на тесте
0                 Random Forest     0.453302
3  Градиентный бустинг (тюнинг)     0.445024
1                           SVR     0.347348
2                           KNN     0.273686

Итоговое качество ансамбля моделей (R2): 0.4445

## Часть 2. Target encoding (**всего 5 баллов**)

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [29]:
sales = pd.read_csv('sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [30]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

### Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`. 

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [32]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'] = all_data['item_target_enc'].fillna(0.3343) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.48303869886217643


#### Способ 2

In [35]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'] = all_data['item_target_enc'].fillna(0.3343) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.48303869886217643


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection. 

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях. 

In [47]:
# YOUR CODE GOES HERE
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=False)

# Создаем пустой столбец
all_data['item_target_enc_kfold'] = 0

# Проходим по всем фолдам
for train_idx, val_idx in kf.split(all_data):
    # Получаем тренировочные и валидационные данные
    train_data = all_data.iloc[train_idx].copy()
    val_data = all_data.iloc[val_idx].copy()
    
    # Рассчитываем средние значения целевой переменной для каждого item_id на тренировочной выборке
    item_id_target_mean = train_data.groupby('item_id')['target'].mean()
    
    # Заполняем значения в валидационной выборке
    all_data.loc[val_idx, 'item_target_enc_kfold'] = val_data['item_id'].map(item_id_target_mean)

# Заполняем NaN глобальным средним
all_data['item_target_enc_kfold'] = all_data['item_target_enc_kfold'].fillna(0.3343)

# Рассчитываем корреляцию
encoded_feature = all_data['item_target_enc_kfold'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

C:\Users\krugl\AppData\Local\Temp\ipykernel_1240\237476771.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.08474576 0.02158273        nan ...        nan        nan        nan]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  all_data.loc[val_idx, 'item_target_enc_kfold'] = val_data['item_id'].map(item_id_target_mean)


0.4092355966844065


Ожидаемый ответ 0.4165

### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`. 

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [ ]:
# Создаем новый столбец для хранения результатов LOO
all_data['item_target_enc_loo'] = 0.0

# Рассчитываем сумму целевой переменной для каждого item_id
item_sum = all_data.groupby('item_id')['target'].transform('sum')

# Рассчитываем количество элементов для каждого item_id
item_count = all_data.groupby('item_id')['target'].transform('count')

# Вычитаем каждое значение из своей группы и делим на (количество-1)
all_data['item_target_enc_loo'] = (item_sum - all_data['target']) / (item_count - 1)

# Заполняем NaN глобальным средним
all_data['item_target_enc_loo'] = all_data['item_target_enc_loo'].fillna(0.3343)

# Рассчитываем корреляцию
encoded_feature = all_data['item_target_enc_loo'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4803848311293093


Ожидаемый ответ 0.4803

### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конктертной категории, а не количество строк в датасете.

In [ ]:
# Определяем параметры сглаживания
alpha = 100
global_mean = 0.3343

# Создаем новый столбец для хранения результатов сглаживания
all_data['item_target_enc_smoothing'] = 0.0

# Рассчитываем среднее целевой переменной для каждого item_id
# item_means = all_data.groupby('item_id')['target'].transform('mean')

# Рассчитываем количество элементов для каждого item_id
item_counts = all_data.groupby('item_id')['target'].transform('count')

item_sum = all_data.groupby('item_id')['target'].transform('sum')

# Применяем формулу сглаживания
all_data['item_target_enc_smoothing'] = (item_sum + global_mean * alpha) / (item_counts + alpha)

# Заполняем NaN глобальным средним
all_data['item_target_enc_smoothing'] = all_data['item_target_enc_smoothing'].fillna(global_mean)

# Рассчитываем корреляцию
encoded_feature = all_data['item_target_enc_smoothing'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.4818198797097257


Ожидаемый ответ 0.4818

### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [45]:
# Создаем новый столбец для хранения результатов expanding mean
all_data['item_target_enc_expanding'] = 0.0


# Рассчитываем кумулятивную сумму таргета для каждого item_id
cum_sum = all_data.groupby('item_id')['target'].cumsum() - all_data['target']

# Рассчитываем кумулятивное количество для каждого item_id
cum_count = all_data.groupby('item_id').cumcount()

# Применяем формулу expanding mean: кумулятивная сумма / кумулятивное количество
all_data['item_target_enc_expanding'] = cum_sum / cum_count

# Заполняем NaN значения глобальным средним
all_data['item_target_enc_expanding'] = all_data['item_target_enc_expanding'].fillna(0.3343)

# Рассчитываем корреляцию
encoded_feature = all_data['item_target_enc_expanding'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)

0.5025245211081716


Ожидаемый ответ 0.5025